In [1]:
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

In [2]:
def calculate_time_domain_features(df):
    features = [
        df['accelerometer_X'].mean(),
        df['accelerometer_Y'].mean(),
        df['accelerometer_Z'].mean(),
        df['accelerometer_X'].std(),
        df['accelerometer_Y'].std(),
        df['accelerometer_Z'].std(),
        df['accelerometer_X'].min(),
        df['accelerometer_Y'].min(),
        df['accelerometer_Z'].min(),
        df['accelerometer_X'].max(),
        df['accelerometer_Y'].max(),
        df['accelerometer_Z'].max()
    ]
    return features

def combine_csv_files(file_list):
    df_list = []

    for file in file_list:
        df = pd.read_csv(file)
        features = calculate_time_domain_features(df)
        df['mean_X'] = features[0]
        df['mean_Y'] = features[1]
        df['mean_Z'] = features[2]
        df['std_X'] = features[3]
        df['std_Y'] = features[4]
        df['std_Z'] = features[5]
        df['min_X'] = features[6]
        df['min_Y'] = features[7]
        df['min_Z'] = features[8]
        df['max_X'] = features[9]
        df['max_Y'] = features[10]
        df['max_Z'] = features[11]
        df_list.append(df)

    combined_df = pd.concat(df_list, ignore_index=True)
    return combined_df

In [3]:
path_idle = '/content/drive/MyDrive/Colabs/data_hw5/idle/*.csv'
path_running = '/content/drive/MyDrive/Colabs/data_hw5/running/*.csv'
path_stairs = '/content/drive/MyDrive/Colabs/data_hw5/stairs/*.csv'
path_walking = '/content/drive/MyDrive/Colabs/data_hw5/walking/*.csv'

idle_file_list = glob.glob(path_idle)
running_file_list = glob.glob(path_running)
stairs_file_list = glob.glob(path_stairs)
walking_file_list = glob.glob(path_walking)

idle_combined_df = combine_csv_files(idle_file_list)
running_combined_df = combine_csv_files(running_file_list)
stairs_combined_df = combine_csv_files(stairs_file_list)
walking_combined_df = combine_csv_files(walking_file_list)

print(idle_combined_df.head())
print(running_combined_df.head())
print(stairs_combined_df.head())
print(walking_combined_df.head())

   accelerometer_X  accelerometer_Y  accelerometer_Z    mean_X    mean_Y  \
0       -10.448297         0.852336         4.553772 -0.575087  5.054958   
1        -2.398990         4.352659         9.179369 -0.575087  5.054958   
2        -2.389413         5.995081         6.052542 -0.575087  5.054958   
3         0.196325         5.300763         8.236054 -0.575087  5.054958   
4         0.301669         5.401319         7.967903 -0.575087  5.054958   

     mean_Z     std_X     std_Y     std_Z      min_X     min_Y     min_Z  \
0  8.051541  1.974139  1.004131  0.913991 -10.448297  0.852336  4.553772   
1  8.051541  1.974139  1.004131  0.913991 -10.448297  0.852336  4.553772   
2  8.051541  1.974139  1.004131  0.913991 -10.448297  0.852336  4.553772   
3  8.051541  1.974139  1.004131  0.913991 -10.448297  0.852336  4.553772   
4  8.051541  1.974139  1.004131  0.913991 -10.448297  0.852336  4.553772   

     max_X     max_Y     max_Z  
0  0.54109  6.627151  9.179369  
1  0.54109  6.627151

In [4]:
time_domain_features = [
    idle_combined_df[['mean_X', 'mean_Y', 'mean_Z', 'std_X', 'std_Y', 'std_Z', 'min_X', 'min_Y', 'min_Z', 'max_X', 'max_Y', 'max_Z']],
    running_combined_df[['mean_X', 'mean_Y', 'mean_Z', 'std_X', 'std_Y', 'std_Z', 'min_X', 'min_Y', 'min_Z', 'max_X', 'max_Y', 'max_Z']],
    stairs_combined_df[['mean_X', 'mean_Y', 'mean_Z', 'std_X', 'std_Y', 'std_Z', 'min_X', 'min_Y', 'min_Z', 'max_X', 'max_Y', 'max_Z']],
    walking_combined_df[['mean_X', 'mean_Y', 'mean_Z', 'std_X', 'std_Y', 'std_Z', 'min_X', 'min_Y', 'min_Z', 'max_X', 'max_Y', 'max_Z']]
]
activity_labels = [0, 1, 2, 3]

X = pd.concat(time_domain_features, ignore_index=True)
y = np.repeat(activity_labels, [df.shape[0] for df in time_domain_features])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [5]:
# SVM classifier
svm_classifier = SVC(kernel='rbf', C=10)
svm_classifier.fit(X_train, y_train)

svm_predictions = svm_classifier.predict(X_test)

svm_accuracy = accuracy_score(y_test, svm_predictions)
svm_classification_report = classification_report(y_test, svm_predictions)
svm_confusion_matrix = confusion_matrix(y_test, svm_predictions)

print("SVM Classifier Results:")
print("Accuracy:", svm_accuracy)
print("Classification Report:")
print(svm_classification_report)
print("Confusion Matrix:")
print(svm_confusion_matrix)

SVM Classifier Results:
Accuracy: 0.9966341689879294
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12374
           1       1.00      1.00      1.00     40827
           2       0.94      0.93      0.93      2014
           3       0.99      0.99      0.99     22329

    accuracy                           1.00     77544
   macro avg       0.98      0.98      0.98     77544
weighted avg       1.00      1.00      1.00     77544

Confusion Matrix:
[[12374     0     0     0]
 [    0 40827     0     0]
 [    0     0  1866   148]
 [    0     0   113 22216]]


In [6]:
# Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

rf_predictions = rf_classifier.predict(X_test)

rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_classification_report = classification_report(y_test, rf_predictions)
rf_confusion_matrix = confusion_matrix(y_test, rf_predictions)

print("Random Forest Classifier Results:")
print("Accuracy:", rf_accuracy)
print("Classification Report:")
print(rf_classification_report)
print("Confusion Matrix:")
print(rf_confusion_matrix)

Random Forest Classifier Results:
Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12374
           1       1.00      1.00      1.00     40827
           2       1.00      1.00      1.00      2014
           3       1.00      1.00      1.00     22329

    accuracy                           1.00     77544
   macro avg       1.00      1.00      1.00     77544
weighted avg       1.00      1.00      1.00     77544

Confusion Matrix:
[[12374     0     0     0]
 [    0 40827     0     0]
 [    0     0  2014     0]
 [    0     0     0 22329]]


Висновок:

- Класифікатор SVM досяг точності 0.9966, з високою точністю, відгуком та F1-оцінкою для кожного класу.
- Класифікатор Random Forest показав ідеальну точність 1.0, що свідчить про те, що він правильно класифікував всі екземпляри для даного набору даних.
- Обидва класифікатори успішно виконали завдання точного передбачення міток активностей на основі даних акселерометра.
- Random Forest є швидкою моделлю навчання, оскільки вона базується на розбитті даних на різні дерева. SVM трохи повільніша через обчислювально витратний процес знаходження оптимальної розділяючої гіперплощини.
- Отримані результати підтверджують, що класифікатор Random Forest показав трохи кращі результати за точністю та швидкістю порівняно з класифікатором SVM, але обидві моделі проявили високу ефективність у класифікації активностей.